In [82]:
from datetime import datetime, timedelta
from lxml import html
import requests
import time
import pandas as pd
import os
import urllib3


__author__ = 'aituarov'


DB_LOADER_DIR = 'C:\\DEV\\output\\'
DATAFILES_DIR = 'C:\\DEV\\output\\'

post_fix = '_ntc_trans'

end_date = datetime.today().date() + timedelta(days=-1)
start_date = end_date + timedelta(days=-1)

# end_date = datetime.strptime("2020-07-11", '%Y-%m-%d').date()
# start_date = datetime.strptime("2020-07-10", '%Y-%m-%d').date()


def download_file():
    urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
    url = 'https://damas.terna.it/api/Ntc/DownloadNtcReport?businessDayFrom='+(start_date + timedelta(days=-1)).strftime('%Y-%m-%d')+'T22:00:00.000Z&businessDayTill='+(end_date + timedelta(days=-1)).strftime('%Y-%m-%d')+'T22:00:00.000Z'
    response = requests.get(url, stream=True, verify=False)
    
    file_name = 'NtcReport_' + datetime.today().strftime('%Y%m%d_%H%M') + '.xlsx'
    with open(DATAFILES_DIR + file_name, 'wb') as output:
            output.write(response.content)
            print('File ' + file_name + ' downloaded')
            output.close()
    
    
def gen_output_csv():
    file_names = [f for f in os.listdir(DATAFILES_DIR) if f.startswith('NtcReport_')]
    for f_id, file_name in enumerate(file_names):
        print("Reading " + file_name)
        data = pd.read_excel(DATAFILES_DIR + file_name)
        data['date'] = data['Business Day'] + '_' +(data['Hour'] - 1).astype(str)
        data = data.drop(columns=['Business Day', 'Hour'])
        
        write_stdldr(data,f_id)
              
        
def write_stdldr(data,f_id):
    print("Writing to new csv file...")
    
    data['name'] = data['Border Direction'] + post_fix
    data['effective_date'] = ""
    data['start_date'] = data['date'].map(lambda x: datetime.strptime(x, '%d.%m.%Y_%H').strftime('%Y-%m-%d_%H:%M'))
    data['end_date'] = ""
    data['value'] = data['NTC [MW]'] 
    data['table'] = 'spot'

    data = data[["name","effective_date","start_date","end_date","value","table"]]

    outfile = DB_LOADER_DIR + "ntc_("+str(f_id)+")_"+ datetime.today().strftime('%Y_%m_%d_%H%M%S') +".csv"
    data.to_csv(outfile, sep=';',index=False)
    print(outfile + " is ready")
            
    
def main():
    download_file()
    time.sleep(3)
    gen_output_csv()



if __name__ == '__main__':

    main()


File NtcReport_20200711_1939.xlsx downloaded
Reading NtcReport_20200711_1936.xlsx
Writing to new csv file...
C:\DEV\output\ntc_(0)_2020_07_11_193936.csv is ready
Reading NtcReport_20200711_1939.xlsx
Writing to new csv file...
C:\DEV\output\ntc_(1)_2020_07_11_193936.csv is ready


In [73]:
# end_date = datetime.strptime("2019-05-30", '%Y-%m-%d').date()
# end_date.strftime('%Y-%m-%d')
# print(datetime.today().strftime('%Y%m%d_%H:%M'))
# print(datetime.today().strftime('%Y%m%d_%H%M') + '.xlsx')
a = '10.07.2020_1'
b = datetime.strptime(a, '%d.%m.%Y_%H').strftime('%Y-%m-%d_%H:%M')
print(b)

ValueError: '%' is a bad directive in format '%d.%m.%Y_%(H-1)'

In [6]:
a = b'PK\x03\x04\x14\x00\x08\x00\x08\x00P=\xebP\xe4H\xad\xaf\x18\x01\x00\x003\x03\x00\x00\x13\x00\x00\x00[Content_Types].xml\xb5\x92\xcfJ\x031\x10\xc6_e\xc9U\x9a\xb4\x1eD\xa4\xdb\x1e\xaa\x1eU\xb0>\xc0\x98\xccvC\xf3\x8f\xcc\xb4\xb6oo6+"\xa5\x82\x1ez\x9a$\xdf\xcc\xf7\xfd\x083_\x1e\xbck\xf6\x98\xc9\xc6\xd0\x8a\x99\x9c\x8a\x06\x83\x8e\xc6\x86M+\xde\xd6\x8f\x93[\xd1\x10C0\xe0b\xc0V\x1c\x91\xc4r1_\x1f\x13RSf\x03\xb5\xa2gNwJ\x91\xee\xd1\x03\xc9\x980\x14\xa5\x8b\xd9\x03\x97k\xde\xa8\x04z\x0b\x1bT\xd7\xd3\xe9\x8d\xd210\x06\x9e\xf0\xe0!\x16\xf3{\xec`\xe7\xb8Y\x8d\xef\x83u+ %g5p\xc1R\xc5L4\x0f\x87"\x8e\x94\xc3]\xfdan\x1f\xcc\t\xcc\xe4\x0bDft\xb5\x87z\x9b\xe8\xea4\xa0\xa84$<\x97\x8f\xc9\xd6\xe0\xbf"b\xd7Y\x8d&\xea\x9d/#\x92RF0\xd4#\xb2w\xb2V\xe9\xc1\x861\xf4\x052?\x81/\xae\xea\xe0\xd4G\xcc\xdb\xf7\x18\xb7\xb2j\x17\x01\x18"\xea\xf9\xb7\xfc*\x92\xaaevA\x10\xe2\xa3C:G1*\x97\x8c\xee!\xa3y\xe5\\\x96\xfc<\xc1\xcf\x86o\x10U\x97~\xf1\tPK\x07\x08\xe4H\xad\xaf\x18\x01\x00\x003\x03\x00\x00PK\x03\x04\x14\x00\x08\x00\x08\x00P=\xebP\x98\xda\xeb\x8b\xae\x00\x00\x00\'\x01\x00\x00\x0b\x00\x00\x00_rels/.rels\x8d\xcf\xc1\x0e\x820\x0c\x06\xe0WYz\x97\x81\x07c\x0c\x83\x8b1\xe1j\xf0\x01\xe6V\x06\x01\xd6e\x9b\no\xef\x8eb<xl\xfa\xf7\xfb\xd3\xb2^\xe6\x89=\xd1\x87\x81\xac\x80"\xcb\x81\xa1U\xa4\x07k\x04\xdc\xda\xcb\xee\x08,Di\xb5\x9c\xc8\xa2\x80\x15\x03\xd4Uy\xc5I\xc6t\x12\xfa\xc1\x05\x96\x0c\x1b\x04\xf41\xba\x13\xe7A\xf58\xcb\x90\x91C\x9b6\x1d\xf9Y\xc64z\xc3\x9dT\xa34\xc8\xf7y~\xe0\xfe\xd3\x80\xad\xc9\x1a-\xc07\xba\x00\xd6\xae\x0e\xff\xb1\xa9\xeb\x06\x85gR\x8f\x19m\xfcQ\xf1\x95H\xb2\xf4\x06\xa3\x80e\xe2/\xf2\xe3\x9dh\xcc\x12\n\xbc*\xf9\xe6\xc1\xea\rPK\x07\x08\x98\xda\xeb\x8b\xae\x00\x00\x00\'\x01\x00\x00PK\x03\x04\x14\x00\x08\x00\x08\x00P=\xebP\xd8v\x12\xfd\xe2\x00\x00\x00c\x01\x00\x00\x0f\x00\x00\x00xl/workbook.xml\x8d\x90AN\xc30\x10E\xafb\xcd\xbe\xb1\x1b\x04\xa2Q\x92.`S\t\tV\xec\x8d3i\xac\xda\x9eh\xecP\xee\xc6\x82#q\x05\x9cF\x11]v\xe5\xff5\x7f\xde|\xf9\xf7\xfb\xa7\xde\x7fy\'>\x91\xa3\xa5\xd0\xc0\xb6P 0\x18\xeal860\xa5~\xf3\x08\xfb\xb6>\x13\x9f>\x88N"\xa7C\xac\xb8\x81!\xa5\xb1\x922\x9a\x01\xbd\x8e\x05\x8d\x18\xf2\xac\'\xf6:e\xcbGI}o\r>\x93\x99<\x86$K\xa5\x1e$\xa3\xd3)_\x8a\x83\x1d#,\xb4[Xqd\xd4]\x1c\x10\x93w\x0b\xcak\x1b\xa0\xad\xe7V\xef\x16\xcf\xf1\xbf\xe4l\x85lky5\xbb\xac\xae\xaf\x08\xdac\x03\xe5\xaeP\xf7E\xa9\xb6;\xb1\x11wj5 .\xa1C\x97\xbf\x03\x04W6\x0b>tY\xcf\xcc\x15d\xb43o,\xfa\xc9\xb9\xa7,_\xc3\x0b\xe9ecN\xadU\xda?PK\x07\x08\xd8v\x12\xfd\xe2\x00\x00\x00c\x01\x00\x00PK\x03\x04\x14\x00\x08\x00\x08\x00P=\xebP\x81b\x92\xa2\xd6\x00\x00\x004\x02\x00\x00\x1a\x00\x00\x00xl/_rels/workbook.xml.rels\xad\x91\xcfj\xc30\x0c\x87_\xc5\xe8\xbe8\xe9`\x8cQ\xb7\x971\xe8\xb5\x7f\x1e@\xd8J\x1c\x9a\xd8\xc6\xd2\xda\xe5\xedk6VR(c\x87\x9e\x84d\xf4\xfd>\xac\xe5\xfak\x1c\xd4\x892\xf71\x18h\xaa\x1a\x14\x05\x1b]\x1f:\x03\x87\xfd\xc7\xd3+(\x16\x0c\x0e\x87\x18\xc8\xc0D\x0c\xeb\xd5rK\x03JYa\xdf\'V\x85\x11\xd8\x80\x17IoZ\xb3\xf54"W1Q(/m\xcc#Jis\xa7\x13\xda#v\xa4\x17u\xfd\xa2\xf3\x9c\x01\xb7L\xb5q\x06\xf2\xc65\xa0\xf6S\xa2\xff\xb0c\xdb\xf6\x96\xde\xa3\xfd\x1c)\xc8\x9d\x08}\x8e\xf9\xc8\x9eH\n\x14sGb\xe0:b\xfd]\x9a\xaaPA\xdf\x97Y<R\x86e\x1a\xca_^M~\xfa\xbf\xe2\x9f\x1f\x1a\xef1\x93\xdbI.\x87\x9e[\xcc\xc7\xbf2\xfa\xe6\xda\xab\x0bPK\x07\x08\x81b\x92\xa2\xd6\x00\x00\x004\x02\x00\x00PK\x03\x04\x14\x00\x08\x00\x08\x00P=\xebP\x10\x96/\xdaJ\x1f\x00\x00v"\x01\x00\x18\x00\x00\x00xl/worksheets/sheet1.xml\x8d\x9dAr\x1cWz\x84\xaf\xc2\xc0\xde\x83\xaeW\xd5\xd5\xd5\n\x8a\x13c2&\xec\x9d\xc3\x0b{M\x93\x90\xc8\x90H(\x00H\x9a\xbby\xe1#\xf9\n\x06)\x99\x95\xf92\xdf_\xb9\xb1g&\x13\xe2G\x05"\xb3\xba\x81\xfc\xfb\x7f\xff\xfb\x7f^\xfe\xf5\x1f\x9f~~\xf1\xdb\xdd\xc3\xe3\xc7\xfb\xcf\xdf\xdfL\x7f9\xdd\xbc\xb8\xfb\xfc\xee\xfe\xfd\xc7\xcf?~\x7f\xf3\xeb\xd3\x0f\xff\xb4\xdd\xfc\xf5\xd5\xcb\xdf\xef\x1f~z\xfcpw\xf7\xf4\xe2\xd9\xfe\xf9\xf1\xbb\x87\xefo><=\xfd\xf2\xdd\xed\xed\xe3\xbb\x0fw\x9f\xde>\xfe\xe5\xfe\x97\xbb\xcf\xcf\xda\x0f\xf7\x0f\x9f\xde>=\xff\xd7\x87\x1fo\xef\x7f\xf8\xe1\xe3\xbb\xbb7\xf7\xef~\xfdt\xf7\xf9\xe9\xb6\x9dN\xeb\xed\xc3\xdd\xcfo\x9f\x9e\xff\xa8\xc7\x0f\x1f\x7fy\xbc\xf9\xe3\x9f\x96\xfc\xb3\x1e\x7fy\xb8{\xfb\xfe+\xc2\xa7\x9f\xff\xf8G}z\xfb\xf1\xf3\xcd\xab\x97\xef?>\xff\xd3\xbf\xd0\xbfx\xb8\xfb\xe1\xfb\x9b\xbfM\xdf\xbdY\xb6\xe9\xe6\xc5\xed\xab\x97_\xed\xff\xf1\xf1\xee\xf7G\xf8\xcf/\xbe\xfc]\xfe\xeb\xfe\xfe\xa7/\xff\xe5_\xdf\x7f\x7fs\xfa\xea\xbd\x15\xf3\xdf\xbf\xfe\xf1\xff\xf6\xf0\xe2\xfd\xdd\x0fo\x7f\xfd\xf9\xe9\xdf\xef\x7f\xff\x97\xbb\x8f?~xz\xfe\xd7t\xfe\xfa5\xef\xee\x7f~\xfc\xfa\x7f_|\xfa\xf8\xe5_\xde\xcd\x8bOo\xff\xf1\xf5\xff\xff\xfe\xf1\xfd\xd3\x87\xefo\xdaz\xf3\xe2\xdd\xaf\x8fO\xf7\x9f\xfe\xf3\x8f\xffa\xba\xb9\x85/h\x7f~AK\xbf`\xfe\xf3\x0b\xe6\xf4\x0b\x96?\xbf`9\xf8\x82\xdb?\xfe*_\xff\xdeo\xde>\xbd}\xf5\xf2\xe1\xfe\xf7\x17\x0f_\xd4\xe7\x7f\xda\x97\xff\xf0\xb7\xe7\xbf\xd5\xe3\xd7\xbf\xdb\xf3\xdf\xff\xf1\xf9\x7f\xfd\xed\xd5\xe9\xe5\xedo_\xbe\xf4O\xc7?\xabcb\xc7ku4v\xbcQ\xc7\xfc\xcdq\xfb\xcc\xf4\r\xac}\x03k_\xbf\xa4\xc1\x97,\x1d\x98:\xce\x1d\xd8\xff;\x0c\xf4\x1b\xd4\xda\xe5\xeaq\xe6o8\xf3!\x8e:z\x9c\x19\xff\xc8\x0egNp\x96o8\xcb!\x8e:z\x9c\x05\xfe\xc8\xb9\xc3Y\x12\x9c\xf37\x9c\xf3!\x8e:z\x9c3\xfc\x91\xddW\xbf9\'8\xeb7\x9c\xf5\x10G\x1d=\xce\n\x7fd\xa7\xbdY\x13\x9c\xcb7\x9c\xcb!\x8e:z\x9c\x0b\xfc\x91k\x87sIp\xb6o8\xdb!\x8e:z\x9c\r\xfe\xc8K\x87\xb3%8\xd7o8\xd7C\x1cu\xf48W\xf8#\xb7\x0e\x07\xb5vm\x1eg:\xed\x89x:\x042\x96\x9e\xe8\x9b\xe5\x8bx\xed\x90H\x1c3AJO\xc7Lj\x11\xa6\t\xfe\xd8\xe9\xd4CM\x11\xd4\x9e\xd0\xd3qD\x1b\x8b@QH\xf7)M\xea\x18j\xcf\xe9\xe98\xa8\x8dE\xa00\x8e\'i\xb39\x82\xda\xd3z:\x8ekc\x11(\x0c\xe5\xa9OlR\xc7P{fO\xc7\xa1m,\x02\x85\xd1<\xf5\xb9M\xea\x18jO\xee\xe98\xba\x8dE\xa00\xa0\xa7>\xbdI\x1dC\xed\xf9=\x1d\x07\xb8\xb1\x08\x14\xc6\xf4\xd4g8\xa9c\xa8=\xc5\xa7\xe3\x187\x16\x81\xc2\xb0\x9e\xfa$\'u\x0c\xb5g\xf9t\x1c\xe6\xc6"P\x18\xd9S\x9f\xe7S\x14\xe8m\x0f\xf4v\x1c\xe8\xc6"\xcf\x92\x98\xd9S\x9f\xe8-J\xf4\xb6\'z;Ntc\x11(\xca\xec>\xd1[\x94\xe8\r\x9e\xb9\x83\x87\xee\xe0\xa9\x9b2[\x9e\xbb\xa3Do{\xa2\xb7\xe3D7\x16\x81\xa2\xcc\xee\x13\xbdE\x89\xde\xf6Do\xc7\x89n,\x02E\x99\xdd\'z\x8b\x12\xbd\xed\x89\xde\x8e\x13\xddX\x04\x8a2\xbbO\xf4\x16=\x8a\xb7=\xd1\xdbq\xa2\x1b\xcb\xdaCQ\xa2\xf7L\xf4\xac>/\x03\xa6=\xd0\xdbq\xa0\x1b\x8b0Qd\xf7L\x97\x88i\xcf\xf3v\x9c\xe7\xc6"L\x98\xd8\xf2\r\xb5EL{\x9c\xb7\xe387\x16a\xc2\xc0\x96\xef\xa7k\xc24\xefi>\x1f\xa7\xb9\xb1\xf4L3\xe6u\xff\x80\xc0\xe2\x90i\x0f\xf3\xf98\xcc\x8dE\x980\xae\xfb\xe7\x03\x12\xc7L{\x96\xcf\xc7Yn,\xc2\x84i\xdd?\x1e\x908f\x827Q\x82wQ\xd4"L\x18\xd6\xfd\xd3\x01\x89\xe7\xcb4`\xda\x93|>Nrc\x11&\xcc\xea\xfe\xe1\x80\xc41\xd3\x1e\xe4\xf3q\x90\x1b\x8b0\xd1\xa3y\xffp@\xea\x18j\x0f\xf2\xf98\xc8\x8dE\xa0(\xc8\xfb$\'u\x0c\xb5\'\xf9|\x9c\xe4\xc6"P\xf4h.o\xcd]"\xa8=\xca\xe7\xe3(7\x16\x81\xa2G\xf3>\xcbI\x1dC\xedY>\x1fg\xb9\xb1\x08\x14=\x9a\xf7aN\xea\x10j\xd9\xc3|9\x0esc\xe9\xa1\x16z4\xef\xd3\x9c\xd41\xd4\x9e\xe6\xcbq\x9a\x1b\x8b@\xd1\x9b-}\x9c\x93:\x86\xda\xe3|9\x8esc\x11(z\xb3\xa5\xcfsR\xc7P{\x9e/\xc7yn,\x02Eo\xb6\xf4\x81\xbeD\x81\xbe\xc0[\xe3\xc1{\xe3\xc7\x81\xbe\xd0\x9b-}\xa2/Q\xa2/{\xa2/\xc7\x89n,\x02E\x0f\xdf}\xa2/Q\xa2/{\xa2/\xc7\x89n,\x02Eo\xa7\xf4\x89\xbeD\x89\xbe\xec\x89\xbe\x1c\'\xba\xb1\x08\x14=\x9b\xf7\x89\xbeD\x89\xbe\xec\x89\xbe\x1c\'\xba\xb1\x08\x14\xbd\x9d"?r\x89\x12}\xd9\x13}9Ntc\x11(z;\xa5O\xf4%z<?\xef\x89~>Ntc\xe9\x92\xe8\xf5\x99\x12\xbdc"\xf14 \xda\xe3\xfc|\x1c\xe7\xc6"D\xf4^JO4\x05D{\x96\x9f\x8f\xb3\xdcX\x84\x08\xd3\xba\xfff"qD\xb4\x07\xf9\xf98\xc8\x8dE\x880\xaa\xe5Gxs@\xb4\xa7\xf8\xf98\xc5\x8dE\x88(\xa7{\xa2% \x82\x9fq\x06?\xe4T\x8b\x10aH\xf7O\x05$\x8e\x88\xf6\xfc>\x1f\xe7\xb7\xb1\x08\x11&t\xffH@\xe2\x88h\x0f\xef\xf3qx\x1b\x8b\x10a<\xf7\xcf\x03$\x8e\x88\xf6\xe4>\x1f\'\xb7\xb1\x08\x11fs\xff0@\xe2\x88h\x8f\xed\xf3ql\x1b\x8b\x10\xd1\x83x\xff(@\xea\x00i\xddC{=\x0emc\xe9\x91V\n\xed>\xb5\xd7 \xb5\xd7=\xb5\xd7\xe3\xd46\x16A\xa2\x87\xf0>\xb6\xd7 \xb6\xd7=\xb6\xd7\xe3\xd86\x16A\xa2G\xf0>\xb7\xd7 \xb7\xd7=\xb7\xd7\xe3\xdc6\x16A\xa2\x07\xf0>\xb8\xd7 \xb8\xd7=\xb8\xd7\xe3\xe06\x16A\xa2\xc7o\xf9\xfd\x8b \xb9\xd7=\xb9\xd7\xe3\xe46\x16A\xa2\xb7S\xfa\xe8^\x83\xe8^\xe17T\x82_Q9\x8e\xee\x95\xdeL\xe9\xb3{\r\xb2{\xdd\xb3{=\xcenc\x11$z+\xa5\x0f\xef5\x08\xefu\x0f\xef\xf58\xbc\x8dE\x90\xe8\x8d\x94>\xbd\xd7 \xbd\xd7=\xbd\xd7\xe3\xf46\x16A\xa2\x87\xee>\xbd\xd7 \xbd/{z_\x8e\xd3\xdbXz\xa4\x0b\xfd\x04\xb3O\xefK\x90\xde\x97=\xbd/\xc7\xe9m,\x82D\xcf\xdc}z_\x82\xf4\xbe\xec\xe9}9Noc\x11$\xfa\xe9e\x9f\xde\x97 \xbd/{z_\x8e\xd3\xdbX\x04\x89~v\xd9\xa7\xf7%H\xef\xcb\x9e\xde\x97\xe3\xf46\x96\xadG\xa2\xf4\xee\x89\xf8w\x0b/\xeb\x00j\xcf\xef\xcbq~\x1b\x8b@\xd1\x9b\'=\xd49\x83\xda\x13\xfcr\x9c\xe0\xc6"P\x98\xd1\xf2\xed\xb4fP\xf0\x9b\x86\xc1\xaf\x1a\xaaE\xa00\xa5\xe5\x1b\xea\x92A\xed)~9Nqc\x11(z{\xa4\x87\xda2\xa8=\xc7/\xc79n,\x02\x85I-\xbf\x96y\x8d\xa0\xb6=\xc9\xb7\xe3$7\x96\x1ej\xc3\xac\x96_\xce<eP{\x96o\xc7Yn,\x02\x85i\xdd?\x16l\xfc\x8b*\xd7\xf3\x00jO\xf3\xed8\xcd\x8dE\xa00\xaf\xfb\x07\x03\x12\x0b\xa8=\xcf\xb7\xe3<7\x16\x81\xa2\xa7\xf1\xfe\xd9`\x9b3\xaa=\xd2\xb7\xe3H7\x16\xa1\xa2H\xef3}[2\xaa=\xd3\xb7\xe3L7\x16\xa1\xa2g\xf2>\xd4\xb7sF\xb5\x87\xfav\x1c\xea\xc6"T\xf4X\xde\xa7\xfa\xb6fT{\xaao\xc7\xa9n,BEO\xe6}\xaco\x97\x8c\n~\x8d<\xf8=\xf2\xe3X\xdf\xe8\xe1\xbc\xcf\xf5m\xcb\xa8\xf6\\\xdf\x8es\xddX\x84\x8a\xde]\xe9\x83}\xbbFT\xd7=\xd8\xaf\xc7\xc1n,=\xd5\x95\xde`\xe9\x93\xfdz\xca\xa8\xf6d\xbf\x1e\'\xbb\xb1\x08\x15\xbd\xc7"\xbf}\x9fE\xfbu\x8f\xf6\xebq\xb4\x1b\x8bP\xd1\xdb,}\xb6_\xb3l\xbf\xee\xd9~=\xcevc\x11*J\xef>\xdb\xafY\xb6_\xf7l\xbf\x1eg\xbb\xb1\x08\x15\xa5w\x9f\xed\xd7,\xdb\xaf{\xb6_\x8f\xb3\xddX\x84\x8a\xd2\xbb\xcf\xf6k\x96\xed\xd7=\xdb\xaf\xc7\xd9n,BE\xe9\xddg\xfb5\xcb\xf6\xeb\x9e\xed\xd7\xe3l7\x16\xa1\xa2\xf4\xee\xb3\xfd\x9a=\xb2_\xf7l\xbf\x1eg\xbb\xb1\\{*\xca\xf6\x1e\x8a\xa2\xbd\xad\x83\x9fw^a(\x14,\x85\xd4"P\x14\xde=\xd45\x82\x9aN8\x17J\xf6B\xea\xe9\xb9v\x8f{)\xc8jE\x06\xa3\xa1S\xb0\x1a2\x1e%\xc3\x0c\x97\x89\xc7i\n\xc9\x1a\x90\x1dg\xbc\xf3(\x19\xe6\xb8\xec<H\xad\xc8f \x0b\xf6C\xc6\xa3d\x98\xe52\xf6 \xb5"\x83\r\xd1)\x18\x11\x19\x8f\x92a\x9e\xcb\xe2\x83\xd4\x8a\x0c\x86D\xa7`Id<J\x86\x99.\xb3\x0fR\xdb\xf9:H\xb1\xe9\x04k\xa2S0\'2\x1e%\xc3\\\xd75\xdf\x1a\x92\xc1\xa4\xe8\x14l\x8a\x8cG\xc9\xe8\xa9^6}$Wh0,:\x05\xcb"\xe3Q4\x8a\x7fY\xf6\x91\\\xa1\xc1\xbc\xe8\x14\xec\x8b\x8cG\xd1\xe8\xf9^\xf6}$\x17h\xb8\x1aMf\xa3\xc6#h\xb4\r\xd5\x95\x1fOG\x0b4\x1c\x8f&\xeb\xd1\xa0\x07x?*E\xc0\x03\xd2\x02\r\x8a \xd9\x90\x1a\x8f\xa2\xd1\xe3\xbe4\x01\xcfH\x0b4h\x82dIj<\x8aF\xef\xe7H\x15\xf0\x98\xb4@\x83*H\xf6\xa4\xc6\xa3h\xf4\xa6\x8et\x01OJ\x0b4\xe8\x82dUj<\x8aF\xef\xecH\x19La\x19\xe0\xb44\xd9\x96\x1a\x8f\xa2\xd1\xdb;\xd2\x06S\xd8\x0680M\x16\xa6\xc6\xa3h\x14\xf7\xba\xf0\x0e\xdb\x00g\xa6\xc9\xce\xd4x\x14\x8d\xe2^\xda`\n\xdb\x00\xc7\xa6\xc9\xda\xd4x\x14\x8d\x9f\xfa\x05-l\x03\x98\x9cN\xc1\xe6\xd4y\x04\x8dw\xa5\xd2\x06-l\x03\x18\x9eN\xc1\xf2\xd4y\x14\x8d\xe2^\xda\xa0\x85/\x0b`~:\x05\xfbS\xe7\xe9\x1f{^O42\xd5\x9b\x02\xa4\x9e\x06?\x15\x9d`\x83:\x05#T\xe71d\xd5\x11\x18V\x0b2\xe8\x82`\x89\xea<\x86\xac\xba\x07\xc3jA\x06U\x10\xccQ\x9d\xc7\x90U\xa7aX-\xc8\xa0\t\x82M\xaa\xf3\x18\xb2\xeaJ\x0c\xab\x05\x19\x14A\xb0Lu\x1eCV\x1d\x8ca\xb5 \x83\x1e\x08\xf6\xa9\xcec\xc8\xaa\xdb1\xac\x16dP\x03\xc1J\xd5y\x0cYuF\xa6SG`\xb0T\x9d\x82\xa9\xaa\xf3(\x18\rR\xe5\x81\x83\xd41\x18T@\xb0Wu\x1e\x03V_\x94!yL\x06\r\x10\xacV\x9d\xc7\x90Q\xc6K\x05\x90<&\x83\x06\x08\xb6\xab\xcec\xc8\x0en\xcb\xcc\x11\x194@\xb0`u\x1eCV\x1f\x98!yL\x06\r\x10\xecX\x9d\xc7\x90\xd5WfH\x1e\x93A\x03\x04cV\xe71d\xf5\xa9\x19\x92\xc7d\xd0\x00\xc1\xa2\xd5y\x0cY}o\x86\xe41\x194@0ku\x1eCV\x1f\x9d!yL\x06\r\x10l[\x9d\xc7\x90\xd5\x97g\xe6\xa8\x02`\xdf:\x05\x03W\xe7Q2\xde\xb8J\x07,Q\x07\xc0\xc8u\nV\xae\xcec\xc8\xca\x1b4,\x8f\xc9\xa0\x03\x82\xa9\xab\xf3\x182z\xd7G:`\x89:\x00\xe6\xaeS\xb0wu\x1eCV^\xa3ayL\x86\x07\xc6\x92\x0bcI\x07\xd0\xb2U_p.Q\x07\xc0\xf0u\n\x96\xaf\xcec\xc8\xca\xbb4,\x8f\x1f\x1ca\xfe:\x05\xfbW\xe7\xe9\x9f&^O4r\xd5\xef3T\xd7e\xf03\xe1\t6\xb0S0\x82u\x1eCV\xdd\xa8a\xb5 \x83\x12\x08\x96\xb0\xcec\xc8\xaaK5\xac\x16dP\x02\xc1\x1c\xd6y\x0cYu\xaf\x86\xd51\x19Lb\xa7`\x13\xeb<JF\xcbWy\xd6 \xb5 \x83\x12\x08\xb6\xb1\xcec\xc8\xaa\xdb5\x9d:&\x83\x12\x086\xb2\xcec\xc80\xe5\xe5Y\x83\xd4\x82\x0cJ \xd8\xca:\x8f!\xab\xee\xd8\xb0\xba\xb6Q\x9e\xc1dv\n6\xb3\xcec\xc8\xaak6\xac\x16dxj2\xb95iJ@\xc9\xca\x9b6,\x17hP\x02\xc1\x88\xd6y\x0cZy\xd9\x86\xe5\x02\rZ X\xd3:\x8fA+\xef\xdb\xb0\\\xa0A\r\x04\xb3Z\xe71h\xe5\x95\x1b\x96\x0b4\xe8\x81`_\xeb<\x06\xad\xbcu\xc3\xf2\x18\rf\xb6S\xb0\xb3u\x1eE\xe3\xa9\xad\x1e8=eh\xd0\x04\xc1\xde\xd6y\x0cZy\xf7\x86\xe5\x02\r\xaa \xd8\xdd:\x8fA\xa3\xf7\x84\xa4\x0bH.\xd0\xa0\x0b\x82\xfd\xad\xf3\x18\xb4\xf2\x06\x0e\xcb\x05\x1a\x94A\xb0\xc3u\x1e\x83V^\xc2a\xb9@\x836\x08\xf6\xb8\xcec\xd0\xca{8,\x17hx~8\xb9?\x9c\xb4\x01\x8do\xf5\xb5\xe7\x9a\xb5\x01\xccs\xa7`\x9f\xeb<\x06\xad\xbc\x8d\xc3r\x81\x06m\x10\xect\x9d\xc7\xa0\x95\x17rX.\xd0\xa0\r\x82\xbd\xae\xf3\x18\xb4\xf2N\x0e\xcb\xe3\x07I\x98\xedN\xc1n\xd7y\xfa\xfe~=\xd18W\xbe\xd5\x92\xe9\xee\x04\xdb\xdd)\x18\xef:\x8f\xe1\xaan\xe6L\xc9~w\x82\x01\xef\x14,x\x9d\xc7pa\xd0\xcb7Y2\xe2\x9d`\xc5;\x053^\xe71\\\xd5\xfd\x9c)Y\xf2N0\xe5\x9d\x82-\xaf\xf3\x18\xae\xea\x8a\x0e\xabC.\x88\xff`\xce\xeb<\x86\xab\xba\xa5\xc3\xea\x90\x0b\xb2?X\xf4:\x8f\xe1\xaa.\xea\xb0:\xe4\xc2\xeb\xf3\xc9\xf9y\x13\xfc\xcaU\xdd\xd5au\xc8\x05\xa9\x1f\xecz\x9d\xc7pU\xd7uX\x1drA\xe4\x07\xd3^\xe71\\\xe5\x8d\x1d\x96G`\xb0\xee\x9d\x82y\xaf\xf3(\x18\x8dx\xf5\xa5\xe6\x96$>,|\xa7`\xe2\xeb<\x06\xac\xbc\xb7\xc3\xf2\x10\x0c"?\x98\xf9:\x8f\x01\xa3G\x7f\xc9\xfc-\xc9|X\xfaN\xc1\xd4\xd7y\x0cXy{\x87\xe5!\x18\x84~\xb0\xf6u\x1e\x03V^\xe0ay\x08\x06\xa9\x1f\x0c~\x9d\xc7\x80\x95wxX\x1e\x82A\xec\x07\x9b_\xe71`\xe55\x1e\x96\x87`\x90\xfb\xc1\xec\xd7y\x0cXy\x93\x87\xe5!\x18~\xf2H\xf2\xd1#I\xf0\xf3\xf8W\x92\x7fK\x92\x1f\xb6\xbfS0\xfeu\x1e\x03V\xde\xe7ay\x04\x06\xf3\xdf)\xd8\xff:\x8f\x82\xf1\xc8W\x92\xff\x9a$?,\x80\xa7`\x02\xec<\x06\xac\xbc\xd5\xc3\xf2\x10\x0c\x92?X\x01;\x8f\x01\xa3\x1f\x02K\xf2_\x93\xe4\x87\x1d\xf0\x14\x0c\x81\x9d\xc7\x80\x95w{X\x1e\x82A\xf2\x07[`\xe7\xe9\xcb\xf0\xf5D\x83_\xfd\x16\xe3_\x04=\x8f^\xe9\xc2\x1ex\n\x06\xc1\xcec\xd0\xaa\x1b>\xacVh\x90\xfe\xc1*\xd8y\x0cZu\xc9\x87\xd5\n\r\xf2?\x98\x06;\x8fA\xab\xee\xf9\xb0Z\xa1A\x03\x04\xfb`\xe71h\xd5U\x1fV+4\xfc\x04\xaa\xe4#\xa8L\x07(Zu\xdb\x87\xd51Z;\xe1\xe7P%\x1fDeZ\xa0Gk4\x06\xee\x1f4X\xad\xd0\xe0\xd3\xa8\x82\xa9\xb0\xf3\x18\xb4\xea\xceO\xa7^\x06\xb1\xd6`*\xdc\x82\xa9\xb0\xf3\x182Lz\xfd\xfc\xae\x96\x91\xcd@\x16|0\x95\xf1\x18\xb2\xf2\xe2\x0f\xcb\x05\x1a|:U\xb0\x15v\x1e\x83V\x9e\xfda\xb9@\x83\xcf\xa8\n\xc6\xc2\xcec\xd0\xca\xdb?,\x17h\xf0IU\xc1Z\xd8y\x0cZy\x00\x88\xe5\x02\r>\xb0*\x98\x0b;\x8fA+\xaf\x00\xb1\\\xa0\xc1\xe7V\x05sa\xe71h\xe5) \x96\x0b4\xf8\xf8\xaa`.\xec<\x06\xad\xbc\x07\xc4\xf2\x18\r\xe6\xc2-\x98\x0b;\x8f\xa2\xf1^X\xca\x80\xe4\x02\r?\x990\xf9h\xc2\xa4\x0bx/,e0ee\x00s\xe1\x16\xcc\x85\x9d\xc7\xa0\xd1\xfbA\xd2\x06S\xd6\x060\x17n\xc1\\\xd8y\x0cZy#\x88\xe5\x02\r\xda \x98\x0b;\x8fA+\x0f\x05\xb1\\\xa0A\x1b\x04sa\xe71h\xe5\xb5 \x96\x0b4h\x83`.\xec<\x06\xad<\x19\xc4r\x81\x06m\x10\xcc\x85\x9d\xc7\xa0\x95w\x83X.\x9e$a/\xdc\x82\xbd\xb0\xf3\xf4U\xf4\xba\xf1G\xd3\n\x1a\xa9\xeb:\xfc\xd7\x06u\x10\xec\x85\x9d\xc7\xa0U7\x84X-\xd0\xf0#j\x93\xcf\xa85\x1eE\xa3E\xb0|\xaf\xf1\x87\xd8\x16hP\x07\xc9\'\xd5\x1a\x8fA\xab\xce\x08\xb1Z\xa1A\x1dD\x9fWk\xea@\xd10\xef\xe5\xc9\x83\xf7\xc2\x05\x1a\xd4A\xf2\xa9\xb5n0\xach\xd5!!V+4\xa8\x83\xe4\xb3k\xddbX\xd1\xaaKB\xacVhP\x07\xc9\'\xd8\xba\xc9\xb0\xa2U\xa7\x84X\x9d\x96e\x88\x06u\x90|\x8e\xad\xdb\x0c+ZuK\x88\xd5\n\r\xea \xf98[7\x1aV\xb4\xf2\x98P\'\x17lP\x07\xc9\xc7\xda\xba\xd9\xb0\xb2\x95\xd7\x84:\xb9`\x83>H>\xde\xd6\r\x87\x95\xad<\'\xd4\xc9c6\xd8\x0e\xb7`;\xec<\xcaF\xeb`}-\xcar\xc1\x06\x8d\x10\xcc\x87\x9d\xc7\xb0\x95\x07\x85zy\xcc\x06\x95\x10\x0c\x88\x9d\xc7\xb0Q\xe8K\'\xb0\\\xb0A\'\x04\x13b\xe71l\xe5I\xa1N.\xd8\xa0\x14\x82\x11\xb1\xf3\x18\xb6\xf2\xa6P\'\x17l\xd0\n\xc1\x8c\xd8y\x0c[yT\xa8\x93\x0b6\xa8\x85`H\xec<\x86\xad\xbc*\xd4\xc9\x05\x1b\xf4B0%v\x1e\xc3V\x9e\x15b\xb9b\x83^\x08\xc6\xc4\xcec\xd8\xca\xbbB,Wl\xd0\x0b\xc1\x9c\xd8y\x0c[yX\xa8\xcda/\xc0\xa0\xb8\x05\x83b\xe7Q6Z\x0c\xeb\xab\xd2%\xec\x05\x98\x14\xb7`R\xec<\x86\xad<-\xd4\x96\xf0\xa5\x02\x8c\x8a\x9b\x1b\x0c\x9f\x05NMga\xa3\xe0\x174\xfai\xf3\xe5:"\x83Vp\x83a%S\x93\x92U\xa7\x85:uL\x06\x9d\xe0\x06\xc3J\xa6&%\xabN\x0b\xb1Z\x90A#\xb8\xc1\xb0\x92\xa9I\xc9\xaa\xd3B\xac\x16d\xd0\x07n/\xacdjR\xb2\xea\xb4\x10\xab\x05\x19\xb4\x81\xdb\x0b+\x99\x9a\x94\xac:-\xc4jA\x06]\xe0\xf6\xc2J\xa6&%\xabN\x0b\xb1Z\x90A\x13\xb8\xbd\xb0\x92\xa9I\xc9\xaa\xd3B\xac\xb6k\x1b\x90\xc1\xa6\xb8\xb9\xbd\xb0\x90\x19\x93\x90\xd1jX\x1e;H-\xc8\xa0\x05\xdc^X\xc9\xd4\xa4d\xe5q!\x96\x0b4(\x017\x18V\xb4\xa0\x04h6\xac/FI.\xd0\xa0\x05\xdcbX\xd1\x82\x16\xa0\xdd\xb0\xbe\x16%\xb9@\x83\x1ap\x93aE\x0bj\x80\x86\xc3\xfaR\x94\xe4\x02\rz\xc0m\x86\x15-\xe8\x01\xde\x15K\x11\x90\\\xa0A\x11\xb8\xcd\xb0\xa2\x05E\xc0\xbbbi\x02\x92\x0b4h\x02\xb7\x19V\xb4\xa0\txW,U@r\x81\x06U\xe06\xc3\x8a\x16T\x01\xef\x8a\xa5\x0bH.\xd0\xa0\x0b\xdcfX\xd1\x82.\xe0]\xb1\x94\xc19+\x03\xd8\x157\xb7\x19\x164c\x124\xde\x15K\x1b\xacY\x1b\xc0\xae\xb8\xb9\xcd\xb0\xa2\x05m@\xc3a}\r\xbafm\x00\xbb\xe2\xe66\xc3\x8a\x16\xb4\x01\r\x87\xf5%\xe8\x9a\xb5\x01\xec\x8a\x9b\xdb\x0c+Z\xd0\x064\x1c\xd6W\xa0k\xd6\x06\xb0+nn3\xachA\x1b\xd0pX_\x80\xaeY\x1b\xc0\xae\xb8\xb9\xcd\xb0\xa2\x05m\xc0\xbbbi\x835{Y\x00\xbb\xe2\xe66\xc3\x8a\xa6\xa6U\xd0\xaaSC\xac\x9e\xe7eD\x06e\xe0&\xc3J\xa6&%\xabN\r\xb1Z\x90A\x17\xb8\xc5\xb0\x92\xa9I\xc9\xaaSC\xac\x16dP\x05n0\xacdjR\xb2\xea\xd4\x10\xabc2\xd8\x147\xb7\x17\x162c\x122Z\r\xcbC\x07\xabc2(\x02\xb7\x18V25)Yuj\x88\xd5\x82\x0cz\xc0m\x86\x95LMJ\x86A/\x8f\x1c\xa4\x16dP\x03n5\xacdjR\xb2\xea\xd4\x10\xab\xe7\xcb\xe0Rw\x83mqs\xbba%S\x93\x92U\xa7\x86X-\xc8\xa0\x04\xdcrX\xc9\xd4\xa4d\xe5\xa9!\x96\x0b4(\x017\x1eV\xb4\xa0\x04hB\xac/?I.\xd0\xa0\x05\xdc~X\xd1\x82\x16\xa0\x15\xb1\xbe\xfc$\xb9@\x83\x1ap\x13bE\x0bj\x80\x86\xc4\xfa\xf2\x93?`x\x8c\x06=\xe0V\xc4\x8a\x16\xf4\x00/\x8d\xa5\x08H\x1e\xa3\xc1\xd6\xb8\xb9\x1d\xb1\xa0\x19\x93\xa0\xf1\xd6X\x9a\x80\xe4\x02\r\x9a\xc0-\x89\x15-h\x02^\x1bK\x15\x90\\\xa0A\x15\xb8-\xb1\xa2\x05U\xc0{c\xe9\x02\x92\x0b4\xe8\x02\xb7&V\xb4\xa0\x0bxq,e\xb0ee\x00\x9b\xe3\xe6\xf6\xc4\x8a\x16\x94\x01o\x8e\xa5\r\xb6\xac\r`u\xdc\xdc\xa2X\xd1\x826\xe0\xcf\x12\x966\xd8\xb26\x80\xddqs\x9bbE\x0b\xda\x80?PX\xda`\xcb\xda\x00\x96\xc7\xcd\xad\x8a\x15-h\x03\xfeTai\x83-k\x03\xd8\x1e7\xb7+V\xb4\xa0\r\xf8\xa3\x85\xa5\r\xb6\xac\r`}\xdc\xdc\xb2X\xd1\x826\xe0\xf5\xb1\xb4\xc1\x96\xbd,\x80\xfdqs\xdbbA3\xa6K\x8f\xc6\x1fA\xdc\x93%\xf3\xe3\x06\xf3\xe3\xe6\xa6\xc5\xca\xa5&\xe5\xaaN\r\xb5d}\xdc`}\xdc\xdc\xb2X\xb9\xd4\xa4\\\x18\xf4\xf2M\x96\x8c\x8f\x1b\x8c\x8f\x9b\x1b\x16+\x97\x9a\x94\xab:5\xd4\x92\xedq\x83\xedqs\xbbb\xe5R\x93rU\xa7\x86X\x1drA\xfc\xbbQ\xb1r\xa9I\xb9\xaaSC\xac\x0e\xb9 \xfb\xdd\xa2X\xb9\xd4\xa4\\\xd5\xa9!V\x87\\\x10\xfcnN\xac\\jR\xae\xea\xd4\x10\xabC.H}\xb7%V.5)Wuj\x88\xd5!\x17D\xbe\x1b\x12+\x97\x9a\x94\xab<5\xc4\xf2\x00l\x86\xa9\xf1\xecf\xc4=\x983\xf5`3\x8f\x89\xfb\xc4gy\x086\x01X\x90\xf8\xce\xa4`\xe5\xa9!\x96\x87`\r\xc0\x82\xc8w&\x05\xa3G\xff>\xf3Y\x1e\x82\xcd\x00\x16d\xbe3)Xyj\x88\xe5!\xd8\x02`A\xe8;\x93\x82\x95\xa7\x86X\x1e\x82\x9d\x01,H}gR\xb0\xf2\xd4\x10\xcbC\xb0\x15\xc0\x82\xd8w&\x05+O\r\xb1<\x04\xbb\x00X\x90\xfb\xce\xa4`\xe5\xa9!\x96\x87`\x1b\x80\x05\xc1\xefL\nV\x9e\x1aby\x08v\x05\xb0 \xf9\x9dI\xc1\xcaSC,\x8f\xc0`X<\xbb\xd1\xb0\x80\x19\x93\x80\xf1\'\rK\xf2OI\xf2\xc3\xacxv\x93a\x05\x0b\x92\x9f?gX\x92\x7fJ\x92\x1fF\xc5\xb3\x1b\x0c+X\x90\xfc\xfc)\xc3\x92\xfcS\x92\xfc0)\x9e\xdd\\X\xc1\x82\xe4\xe7I\xb1$\xff\x94$?\x0c\x8ag7\x16V05m\x02V\x9d\x1ab\xb5].\x83O7\x9daP<\xbb\xb1\xb0\xa2\xa9I\xd1\xaaSC\x9dZ\xa0A\xfa\xbb\xb1\xb0\xa2\xa9I\xd1\xaaSC\xacVh\x90\xffn,\xachjR\xb4\xea\xd4\x10\xab\x15\x1a4\x80\xdb\n+\x9a\x9a\x14\xad:5\xc4j\x85\x06\x1d\xe0\xb6\xc2\x8a\xa6&E\xabN\r\xb1Z\xa0\xc1\x9exv[aA3&A\xa3\xc5\xb0<h\xf0\xe7\x0f\x17h\xd0\x03n+\xachjR\xb4\xea\xd4\x10\xab\xcf/\xa0\x06\xdb\xf5\x19\xf6\xc4\xb3\xdb\n+\x9a\x9a\x14\r\xa3^\x1e6H\xad\xd0\xa0\x0b\xdcVX\xd1\xd4\xa4h\xe5\xb1!\x96+6\xa8\x037\x16V\xb6\xa0\x0ex2,}\xd0\x96\x90\r\xfa\xc0\xad\x85\x95-\xe8\x03\xde\x0cK!\xb4s\xc8\x06\x85\xe0\xe6\xc2\xca\x16\x14\x02\x8f\x86\xa5\x11\xda\x1a\xb2A#\xb8\xbd\xb0\xb2\x05\x8d\xd0\x8d\x86\x85\xed\x12\xb2A%\xb8\xbd\xb0\xb2\x05\x95\xc0\xa3a\xe9\x84\xb6\x85l\xd0\tn/\xaclA\'\xf0hXJ\xa1]36\xd8\x14\xcfn/,l\xc6$l<\x1a\x96V\x98O!\x1b\xb4\x82\xdb\x0b+[\xd0\n<\x1a\x96Z\x98\xc3Z\x80M\xf1\xec\xf6\xc2\xca\x16\xd4\x02\x8f\x86\xa5\x17\xe6\xb0\x17`S<\xbb\xbd\xb0\xb2\x05\xbd@\xa3a}\x1d:\x87\xbd\x00\x9b\xe2\xd9\xed\x85\x95-\xe8\x05\x1a\r\xebK\xd19\xec\x05\xd8\x14\xcfn/\xaclA/\xd0hX_\x8d\xcea/\xc0\xa6xv{ae\x0bz\x81F\xc3\xfa\x82t\x0e{\x016\xc5\xb3\xdb\x0b+[\xd0\x0b\xbc)\x96^\x98\xc3\x97\n\xb0)\x9e\xdd^X\xd9\xd4t\x15\xb6\xea\xf4\x10\xab\xad\xad\x83\x9f\r\xcf0)\x9e\xdd\\X\xd1\xd4\xa4h\xd5\xe9\xa1N\x1d\xa3\xc1\xa2xvkaA3&A\xa3\xc9\xb0|\xb3\xf1\xde\xb8@\x83RpcaES\x93\xa2U\xa7\x87X\xad\xd0\xa0\x13\x92=\xb13)\x1a\x86\xbe<\x82\xf0\x9e\xb8@\x83JH\x06\xc5\xce\xa4h\xd5\xe9\xa1\xb9\xfb\x0c\xe31\x1a4B\xb2(v&E\xabN\x0f\xb1Z\xa1A!$\x93bgR\xb4\xea\xf4\x10\xab\xed|\x1d\xe5\x1al\x8a\xe7dS\xecL\x8aV\x9d\x1eb\xb5B\x83:HF\xc5\xce\xa4h\xe5\xe9!\x96+6\xa8\x83dV\xecL\xcaV\x9e\x1eb\xb9b\x83>H\x86\xc5\xce\xa4l\xe5\xe9!\x96\x0b6\x98\x16\xcf\xc9\xb4\xd8\x99\x84\x8d\xc6\xc3\xfa\xb2\x94\xb7\xc5\x05\x1b4B2.v&e+O\x0f\xb1\\\xb1A%$\xebbgR6z\x99 \x9d\xc0\xeb\xe2\x82\r:!\x99\x17;\x93\xb2\x95\xa7\x87X\xae\xd8\xa0\x14\x92}\xb13)[yz\x88\xe5\x8a\rZ!\x19\x18;\x93\xb2\x95\xa7\x87X\xae\xd8\xa0\x16\x92\x85\xb13)[yz\x88\xe5\x8a\rz!\x99\x18;\x93\xb2\x95\xa7\x87:\xb9`\x83^H6\xc6\xce\xa4l\xe5\xe9\xa1N.\xd8\xa0\x17\x92\x91\xb13)[yz\xa8\x93\xc7l\xb02\x9e\x93\x95\xb13\t\x1b\xcf\x88\xa5\x17\xd6\xb0\x17`f<\'3cgR\xb6\xf2\xf4P\'\x8f\x9f,ag<\';ck:\t\x1c\x15\x83\xb0\x91z\x1a\xfd \x17v\xc6s\xb23\xb6&E\xab\x8e\x0f\xb1Z\xa0A+$;ckR\xb4\xea\xfa\x10\xab\x05\x1a\x94B\xb23\xb6&E\xab\xce\x0f\xb1Z\xa0A\'$;ckR\xb4\xea\xfe\x10\xab\x05\x1aTB24\xb6&E\xab\x0e\x10\xb1Z\xa0A#$KckR\xb4\xea\x02\x11\xab\x05\x1a\x14B25\xb6&E\xabN\x10u\xea\x88\x0c\xa6\xc6s25\xb6&!\xa35\xb1<~\x90:&\x832H\xa6\xc6\xd6\xa4d\xe5\t"\x96\xc7h\xd0\x05\xc9\xd6\xd8\x9a\x14\x8d\xd2^\xca\x80\xe41\x1atA26\xb6&E+O\x10\xb1<F\x83.H\xd6\xc6\xd6\xa4h\xe5\t"\x96\xc7h\xd0\x05\xc9\xdc\xd8\x9a\x14\xad<A\xc4\xf2\x18\r\xba \x99\x1b[\x93\xa2\x95\'\x88X\x1e\xa3A\x17$sckR\xb4\xf2\x04\x11\xcbc4\xe8\x82dnlM\x8aV\x9e by\x8c\x06]\x90\xcc\x8d\xadI\xd1\xca\x13D,\x0f\xd1`n<\'sck\x124\xde\x1bK\x1blQ\x1b\xc0\xdcxN\xe6\xc6\xd6\xa4h\xe5\t"\x96\xc7h\xd0\x06\xc9\xdc\xd8\x9a\x14\x8d\xde\x13\x926\xd8\xa26\x80\xb9\xf1\x9c\xcc\x8d\xadI\xd1\xca\x13D,\x8f\xd1\xa0\r\x92\xb9\xb15)Zy\x82\x88\xe51\x1a\xb4A27\xb6&E+O\x10\xb1<~\x92\x84\xbd\xf1\x9c\xec\x8d\xadi\x12\xb6\xea\x06\x11\xab\xeb2\xfa\xa12\xec\x8d\xe7dolM\x8aV\x1d!b\xb5@\x83:H\xf6\xc6\xd6\xa4h\xd5\x15"V\x0b4\xa8\x83dolM\x8aV\x9d!bu\x8c\x06{\xe39\xd9\x1b[\x93\xa0\xd1\xa4X\x9e<H-\xd0\xa0\x0e\x92\xc9\xb15)Zu\x88\xa8S\xc7hP\x07\xc9\xea\xd8\x9a\x14\r\xf3^\x9e<H-\xd0\xa0\x0e\x92\xe1\xb15)Zu\x8a\x88\xd5\xb5\x8dr\r\xb6\xc7s\xb2=\xb6&E\xabn\x11\xb1Z\xa0A\x1d$\xf3ckR\xb4\xf2\x18\x11\xcb\x05\x1b\xd4A2A\xb6&e+\xaf\x11\xb1\\\xb0A\x1f$3dkR\xb6\xf2\x1c\x11\xcb\x05\x1b\x14B2E\xb6&e+\xef\x11\xb1\\\xb0A#$sdkR\xb6\xf2 \x11\xcbC\xb6\x05\x16\xc9K\xb2H\xb6\xa6\x9em\xe1Ir\xdf\t,\x17l\x13\xb0\x05\x9d`M\xcaV\x9e$b\xb9`k\xc0\x16\x94\x825)\x1b\xbdc\xd4\xb7\x02\xcb\x05\xdb\x0clA+X\x93\xb2\x95G\x89X.\xd8\x16`\x0bj\xc1\x9a\x94\xad\xbcJ\xc4r\xc1v\x06\xb6\xa0\x17\xacI\xd9\xca\xb3D,\x17l+\xb0\x05\xbd`M\xcaV\xde%b\xb9`\xbb\x00[\xd0\x0b\xd6\xa4l\xe5a"\x96\x0b\xb6\r\xd8\x82^\xb0&e+/\x13\xb1\\\xb0]\x81-\xe8\x05kR\xb6\xf24\x11\xcb\xc3\'\xcb\x05\xf6\xcaK\xb2W\xb6\xa6\xd6\xb3\xf1\'!\xf7h\xc9^y\x81\xbd\xf2\x92\xec\x95\xadI\xc1\xaa\xe3DK\xb2W^`\xaf\xbc${ekR0\x8c|\xf9FK\xf6\xca\x0b\xec\x95\x97d\xaflM\nV\x9d\'Z\x92\xbd\xf2\x02{\xe5%\xd9+[\x93\x82U\xf7\x89X\x1d\x82A\x11$kekR\xb0\xea@\x11\xabC0h\x81d\xablM\nV](bu\x08\x06\x15\x90,\x95\xadI\xc1\xaa\x13E\xac\x0e\xc1 \xff\x93\x9d\xb25)Xu\xa3\x88\xd5!\x18\x84\x7f\xb2R\xb6&\x05+\x8f\x14\xb1<"\x83\x91\xf2\x92\x8c\x94\xadI\xc8\xf8s\x8d%\xfb[\x92\xfd\xb0Q^\x92\x8d\xb25)Yy\xa6\x88\xe5!\x19\x84\x7f2Q\xb6&%\xa3\x97\x03\x92\xfe-I\x7fX(/\xc9B\xd9\x9a\x94\xac<T\xc4\xf2\x90\x0c\xe2?\xd9\'[\x93\x92\x95\x97\x8aX\x1e\x92A\xfe\'\xebdkR\xb2\xf2T\x11\xcbC2(\x80d\x9blMJV\xde*byH\x06\r\x90,\x93\xadI\xc9\xcacE,\x0f\xc9\xa0\x02\x92]\xb25)Yy\xad\x88\xe5!\x19t@\xb2J\xb6&%+\xcf\x15\xb1<"\x83M\xf2\x92l\x92\xadI\xc8xu,\x1d0\'\x1d\x00\x8b\xe4%Y$[\x93\x92\x95\x07\x8bX\x1e\x92A\x07${dkR2\xfa\xb1\xb1t\xc0\x9ct\x00\xac\x91\x97d\x8dlMJV\x9e,byH\x06\x1d\x90l\x91\xadi\x16\xb2\xeafQ\xa7\x9e\xce\xa3\x97\xc0\xb0E^\x92-\xb25)[u\xb4\x88\xd5\x8a\rz \xd9"[\x93\xb2UW\x8bX\xad\xd8\xa0\t\x92-\xb25)[u\xb6\x88\xd5\x8a\r\xba \xd9"[\x93\xb2Uw\x8bX\xad\xd8\xa0\r\x921\xb25)[u\xb8\x88\xd5\x82\r\xd6\xc8K\xb2F\xb6&a\xa3\xc1\xb1<v\xf0\xe7\x1b\x17l\xd0\x08\xc9\x1c\xd9\x9a\x94\xad:]\xd4\xa9\x97Q\xbc\xc1\x1cyI\xe6\xc8\xd6\xa4h\x98\xf9\xf2\xe4Aj\x81\x06\x9d\x90\xcc\x91\xadI\xd1\xca\xd3E,\x17l\xd0\n\xc9\x1e\xd9\x9a\x94\xad<]\xc4r\xc1\x06\xad\x90\x0c\x92\xadI\xd9\xca\xd3E,\x17l\xd0\n\xc9"\xd9\x9a\x94\xad<]\xc4r\xc1\x06\xad\x90L\x92\xadI\xd9\xca\xd3E,\x17l\xd0\n\xc9$\xd9\x9a\x94\xad<]\xc4r\xc1\x06\xad\x90L\x92\xadI\xd9\xca\xd3E,\x8f\xd9`\x92\xbc$\x93dk\x126\xde$K-\x90\\\xb0A+$\x93dkR\xb6\xf2t\x11\xcb\x05\x1b\xd4B2I\xb6&e\xa3w\x8b\xa4\x17\xceY/\xc0$yI&\xc9\xd6\xa4l\xe5\xe9"\x96\x0b6\xe8\x85d\x92lM\xcaV\x9e.b\xb9`\x83^H&\xc9\xd6\xa4l\xe5\xe9"\x96\x0b6\xe8\x85d\x92lM\xcaV\x9e.b\xb9`\x83^H&\xc9\xd6\xa4l\xe5\xe9"\x96\x8bGK\xd8$/\xc9&\xd9\x9a\x16\x81\xabn\x17u\xea\xba\x0e\xff\xc5A1$\x9bdkR\xb6\xeax\x11\xab\x05\x1bl\x92\x97d\x93lM\xc2F\xabc\xf9~\xe3\x0fF.\xd8\xa0\x18\x92M\xb25)[u\xbe\x88\xd5\x8a\r\x8a!\xd9$[\x93\xb2a\xf2\xcbs\x08o\x92\x0b6(\x86d\x94lM\xcaV\x1d0b\xb5b\x83bHV\xc9\xd6\xa4l\xd5\x05#V+6(\x86d\x96lM\xcaV\x9d0buz~%=b\x83bHv\xc9\xd6\xa4l\xd5\r#V+6(\x86d\x98lM\xcaV\x1e1\xea\xe4\x02\x0e\x8a!\x99&[\x93\xc2\x95W\x8c:\xb9\x80\x83fH\xc6\xc9\xd6\xa4p\xe5\x19\xa3N\x1e\xc3\xc1>yI\xf6\xc9\xd6$p\xb4@\xd6\xd7\xa8,\x17p\xd0\r\xc9D\xd9\x9a\x14\xae<d\xd4\xcbc8(\x87d\xa4lM\nG\xf1/\xed\xc0r\x01\x07\xed\x90\xcc\x94\xadI\xe1\xcaSF\x9d\\\xc0A=$CekR\xb8\xf2\x96Q\'\x17p\xd0\x0f\xc9T\xd9\x9a\x14\xae<f\xd4\xc9\x05\x1c\x14D2V\xb6&\x85+\xaf\x19ur\x01\x07\r\x91\xcc\x95\xadI\xe1\xcasF,Wp\xd0\x10\xc9`\xd9\x9a\x14\xae\xbcg\xc4r\x05\x07\r\x91L\x96\xadI\xe1\xca\x83F\xcb%l\x08\x18-/\xc9h\xd9\x9a\x04\x8eV\xc9\xfaju\x0b\x1b\x02f\xcbK2[\xb6&\x85+O\x1a-\xdb\xc1\xcb\x87\xdb\xc7\x0fwwOo\xde>\xbd}\xf5\xf2\xc3\xdd\xdb\xf7w\x0f\x7f\xbf\xbf\x7f\xba{xq\xfb\xac\xfd~\xff\xf0\xd3W\xfd\xd5\xff\x01PK\x07\x08\x10\x96/\xdaJ\x1f\x00\x00v"\x01\x00PK\x03\x04\x14\x00\x08\x00\x08\x00P=\xebP}v\x9ep\xc9\x01\x00\x00\xa7\x05\x00\x00\r\x00\x00\x00xl/styles.xml\xa5T\xcbN\x840\x14\xfd\x95\xa6{\xa7\x033N\x8c\x01&q\x12\x127nt\xe1\xb6@\x81&}\x90\xb6\x18\xc6_s\xe1\'\xf9\x0b\xb6\x85\x19\x90`\xcc8lz{z\xcf\xb9\xaf\x96\xaf\x8f\xcfh\xdfq\x06\xde\x88\xd2T\x8a\x18\x06\xab5\x04D\xe4\xb2\xa0\xa2\x8aak\xca\x9b;\xb8O"m\x8e\x8c<\xd7\x84\x18`\xfd\x85\x8eamLs\x8f\x90\xcek\xc2\xb1^\xc9\x86\x08{RJ\xc5\xb1\xb1[U!\xdd(\x82\x0b\xedH\x9c\xa1p\xbd\xde!\x8e\xa9\x80I$Z\x9er\xa3A.[abhC\xa2$*\xa5\x18\xa1\r\xec\x01\x1b\xf9\x1d\xbcaf3\x0b\xbc\x97\xc0\x9c\xf4\xc0\x013\x9a)\xeaQ\xd4\xfb.2r\xc9\xa4\x02\xaa\xcab\x98\x0e\x1f\x04\x86\x8e\x81\xaf\x94\\\xfb\xef\x02I\xbfh+M\x19;\x17\xbc\x85=\x90D\r6\x86(\x91\xda\r\x18\xec\x97cCb(\xa4 \x83\x8ew\xfc\xc3\xbdR\xf8\x18\x84\xb7\x170\xb4d\xb4pyT\x87i\x81\xdb\xddv7\xef\x19\x9a\x88\\)\xbf4\x92Ey\xbf\xd8\xbeeR\x15\xf6\xbe\x9e:\x17\xc2\x13\x94D\x8c\x94\xc6\xf1\x15\xadjo\x18\xd9\xb8%\x93\xc6H\xee\xac\x82\xe2J\n\xcc|\x94\x13\xed\x07\xdd_\xf5\x18\x9a\xda^\xd5Qk\x8e:\xe196D\x99\xc3\x8b!\x07\xc3\x96\x93\x13\xc6\x9e\x1d\xe5\xb5<\xd7\x14\xd8\x9a\xba\x12\xf4\xef\xe4\xb1\xf0mq\xb7f0\x9d\xd0\x94\xd6\x8bL\xf8\x9b\x05>n\x1av|jyFT\xea_\xa9\x0b3U\xf5\x0e\xa9\xec\xe3\x83\xae\x1c\x83]&\x15\xcc\xa5\xdc\xdc\xdcAx:\xb0{\x7f\xd0\xf7`\xcay\xf0\xc8\x95\t\x84\xbf%\xb0\xf9w\x02hh\xf0d\\?\x86uF\x81{\xfa1|rY\xb1\x89F\xd6Rfo\xf8\xd2\x00\xadh\xd1\x95\xb3\xbf!\x1a\xff\xb8\xc97PK\x07\x08}v\x9ep\xc9\x01\x00\x00\xa7\x05\x00\x00PK\x03\x04\x14\x00\x08\x00\x08\x00P=\xebP\xee\xc5i\xf6\xff\x00\x00\x00;\x02\x00\x00\x14\x00\x00\x00xl/sharedStrings.xmlu\xd1AO\xc3 \x14\x07\xf0\xafB\xb8\xb7\xb0\x1a\x17g\xda.s\xab\xee\xa0\xd3\xac5;,;\x90\xf6\xb9\x92\xb4Py\xd4\xb8o/5\xc6\x03\xe2\x91\xff/\xe4\xfdy\xa4\xcb\xcf\xbe#\x1f`Pj\x95\xd1Y\xcc)\x01U\xebF\xaasF_\xab\xfb\xe8\x86\x12\xb4B5\xa2\xd3\n2z\x01\xa4d\x99\xa7\x88\x96\xb8\xab\n3\xdaZ;\xdc2\x86u\x0b\xbd\xc0X\x0f\xa0\x9c\xbci\xd3\x0b\xeb\x8e\xe6\xccp0 \x1al\x01l\xdf\xb1\x84\xf39\xeb\x85T\x94\xd4zT\xd6\x8d\x9dS2*\xf9>\xc2\xfa7p#d\x9e\xda\xfcnD\xa9\x00\x91l\xc4%e6O\xd9\x94\xff\x986\r\x18\xb2\x91\x06j\xeb\x1e\xe0\xfbV\x8f\xc6\xcfv\xd5\x9a\x1c\x9f\x0e\'?O\x161\xbf\x8e\x13>[\xf8\xb2zy\x88\xaab\xbf[\xf9P<\x16eX\xb6U\xf9\x1c\x96}U\x84\xa1<\xfc3\xe5;\x8c\\\x890L%\xc22\x95\x08\x8b+\x11\x06W\xc2\x87+\xfew/\xcc\xfd~\xfe\x05PK\x07\x08\xee\xc5i\xf6\xff\x00\x00\x00;\x02\x00\x00PK\x01\x02-\x00\x14\x00\x08\x00\x08\x00P=\xebP\xe4H\xad\xaf\x18\x01\x00\x003\x03\x00\x00\x13\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00[Content_Types].xmlPK\x01\x02-\x00\x14\x00\x08\x00\x08\x00P=\xebP\x98\xda\xeb\x8b\xae\x00\x00\x00\'\x01\x00\x00\x0b\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00Y\x01\x00\x00_rels/.relsPK\x01\x02-\x00\x14\x00\x08\x00\x08\x00P=\xebP\xd8v\x12\xfd\xe2\x00\x00\x00c\x01\x00\x00\x0f\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00@\x02\x00\x00xl/workbook.xmlPK\x01\x02-\x00\x14\x00\x08\x00\x08\x00P=\xebP\x81b\x92\xa2\xd6\x00\x00\x004\x02\x00\x00\x1a\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00_\x03\x00\x00xl/_rels/workbook.xml.relsPK\x01\x02-\x00\x14\x00\x08\x00\x08\x00P=\xebP\x10\x96/\xdaJ\x1f\x00\x00v"\x01\x00\x18\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00}\x04\x00\x00xl/worksheets/sheet1.xmlPK\x01\x02-\x00\x14\x00\x08\x00\x08\x00P=\xebP}v\x9ep\xc9\x01\x00\x00\xa7\x05\x00\x00\r\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\r$\x00\x00xl/styles.xmlPK\x01\x02-\x00\x14\x00\x08\x00\x08\x00P=\xebP\xee\xc5i\xf6\xff\x00\x00\x00;\x02\x00\x00\x14\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x11&\x00\x00xl/sharedStrings.xmlPK\x05\x06\x00\x00\x00\x00\x07\x00\x07\x00\xc2\x01\x00\x00R\'\x00\x00\x00\x00'
print(a.decode('utf-8'))

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xeb in position 12: invalid continuation byte